In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### Pretrain datasets

In [2]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.07s/it]

CPU times: user 37.6 s, sys: 45.1 s, total: 1min 22s
Wall time: 56.3 s


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np

import bisect

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

df_public = pq.read_table('data/public_train.pqt').to_pandas().sort_values(by='user_id')

# Combined target age_gender
df_public['age'] = list(map(age_bucket, df_public['age'] ))
df_public = df_public[(df_public['age'] != 'NA') & (df_public['is_male'] != 'NA')]
df_public = df_public.dropna()
df_public['target'] = df_public['age'].astype(int) + 7 * df_public['is_male'].astype(int)

# Merge
df_supervised = df_trans.merge(df_public[['user_id', 'target']], on='user_id')

df_train_trans, df_valid_trans = train_test_split(df_supervised, test_size = 0.1, random_state = 3)

df_train_trans = df_train_trans.to_dict(orient='records')
df_valid_trans = df_valid_trans.to_dict(orient='records')

In [4]:
len(df_train_trans), len(df_valid_trans)

(237893, 26433)

## Train COLES

### Model definition

In [5]:
import torch
import pytorch_lightning as pl
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import ColesSupervisedModule
from ptls.nn import L2NormEncoder

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'request_cnt': 'identity'},
    embeddings={ 
        'part_of_day': {'in': 4, 'out': 1},
        'url_host': {'in': 132025, 'out': 512}
    }
)


seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=512,
    type='gru',
)

model = ColesSupervisedModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.9),
    l_loss = 'centroid_softmax',
    contrastive_loss_w=0.1,
    supervised_loss_w=0.01
)

### Dataloader

In [6]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.frames.coles import ColesSupervisedIterableDataset
from ptls.data_load.augmentations import AllTimeShuffle, DropoutTrx
from  ptls.data_load.datasets import AugmentationDataset


train_ds = MemoryMapDataset(data=df_train_trans)
valid_ds = MemoryMapDataset(data=df_valid_trans)

train_ds = AugmentationDataset(train_ds, f_augmentations = [AllTimeShuffle(), DropoutTrx(trx_dropout=0.01)])

train_data=ColesSupervisedIterableDataset(train_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200), cols_classes='target')
valid_data=ColesSupervisedIterableDataset(valid_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200), cols_classes='target')

dl = PtlsDataModule(
    train_data=train_data, train_num_workers=16, train_batch_size=256, 
    valid_data=valid_data, valid_num_workers=16, valid_batch_size=256
)

### Trainer

In [7]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=50,
    limit_val_batches=100,
    gpus=[0],
    enable_progress_bar=False,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, dl)
print(trainer.logged_metrics)

logger.version = 80


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 69.2 M
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
69.2 M    Trainable params
0         Non-trainable params
69.2 M    Total params
276.705   Total estimated model params size (MB)
/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:152: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/p

In [ ]:
torch.save(model.state_dict(), "coles-sup-emb.pt")

# Inference

In [ ]:
model.load_state_dict(torch.load("coles-sup-emb.pt"))

In [ ]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

In [ ]:
%%time
import tqdm
from ptls.data_load.datasets import inference_data_loader
import numpy as np

def pooling_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            x = model.seq_encoder.trx_encoder(batch.to(device)).payload
            out_max = torch.max(x, dim=1)[0]
            out_min = torch.min(x, dim=1)[0]
            out_mean = torch.mean(x, dim=1)
            out_std = torch.std(x, dim=1)
            features = torch.cat([out_max, out_min, out_mean, out_std], dim=1)      
            X += [features]
    return X

def embed_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            features = model.seq_encoder(batch.to(device))
            X += [features]
    return X

dl = inference_data_loader(df_trans.to_dict(orient='records'), num_workers=0, batch_size=128)
X_coles = torch.vstack(embed_inference(model, dl, )).cpu().numpy()
X_pool = torch.vstack(pooling_inference(model, dl, )).cpu().numpy()
X_embeds = np.concatenate([X_coles, X_pool], axis=1)


df_embeds = pd.DataFrame(X_embeds, columns=[f"embed_{e}" for e in range(X_embeds.shape[1])])
df_embeds['user_id'] = df_trans['user_id']
df_embeds.to_csv('./data/coles_sup.csv', index=False)

## Downstream

In [ ]:
%%time

import bisect
import numpy as np

df_embeds = pd.read_csv('./data/coles_sup.csv')
df_public = pq.read_table('data/public_train.pqt').to_pandas().sort_values(by='user_id')

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

y_age = df_public['age']
y_age = np.array(list(map(age_bucket, y_age)))
y_gender = np.array(df_public['is_male'])

X = df_public
X = X.merge(df_embeds, on="user_id", how='inner')
del X['user_id'], X['age'], X['is_male']

## Gender

In [ ]:
# %%time

from catboost import CatBoostClassifier, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
not_na_gender = (y_gender != 'NA') & (y_gender != None)
x_train, x_test_gender, y_train, y_test_gender = train_test_split(X[not_na_gender], y_gender[not_na_gender], test_size = 0.1, random_state = 42)

clf_gender = CatBoostClassifier(
    iterations=1000,
    custom_metric=[metrics.AUC()],
    use_best_model=True,
    random_seed=42)
clf_gender.fit(x_train, y_train, metric_period=100, eval_set=(x_test_gender, y_test_gender))

In [21]:
print(f'GINI по полу {2 * roc_auc_score(y_test_gender, clf_gender.predict_proba(x_test_gender)[:,1]) - 1:2.3f}')

GINI по полу 0.720


# Age

In [ ]:
%%time

from sklearn.metrics import classification_report

not_na_age = ~np.isnan(y_age)
x_train, x_test_age, y_train, y_test_age = train_test_split(X[not_na_age], y_age[not_na_age], test_size = 0.1, random_state = 42)

clf_age = CatBoostClassifier(iterations=1000,
    custom_metric=[metrics.Accuracy()],
    use_best_model=True,
    random_seed=42)
clf_age.fit(x_train, y_train, metric_period=100, eval_set=(x_test_age, y_test_age))

In [20]:
print(classification_report(y_test_age, clf_age.predict(x_test_age), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

              precision    recall  f1-score   support

         <18       0.00      0.00      0.00       109
       18-25       0.53      0.37      0.44      3238
       25-34       0.50      0.64      0.56      8863
       35-44       0.41      0.50      0.45      7773
       45-54       0.38      0.23      0.29      4218
       55-65       0.40      0.24      0.30      2254
         65+       0.30      0.02      0.03       545

    accuracy                           0.45     27000
   macro avg       0.36      0.29      0.30     27000
weighted avg       0.44      0.45      0.44     27000



In [ ]:
0.752 + 2*0.46

In [ ]:
0.79 + 2*0.49